In [1]:
from enum import Enum
from random import random
import numpy as np

In [2]:
class Action(Enum):
    A = 0
    B = 1


def choose_action():
    if 7 * random() < 1:
        return Action.A
    else:
        return Action.B


In [3]:
for i in range(20):
    print(choose_action().name, end=', ')

B, B, B, B, B, B, B, B, B, A, B, B, B, B, B, B, B, B, B, B, 

In [4]:
class Problem:
    class State(Enum):
        S1 = 0
        S2 = 1
        S3 = 2
        S4 = 3
        S5 = 4
        S6 = 5
        S7 = 6

    STATES = list(State.__members__.values())
    N_STATES = len(STATES)

    def __init__(self):
        self._transition_probs = self.define_transition_probabilities()

    @property
    def transition_probs(self):
        return self._transition_probs

    @property
    def transition_probs_a(self):
        return self._transition_probs[Action.A.value]

    @property
    def transition_probs_b(self):
        return self._transition_probs[Action.B.value]

    @classmethod
    def define_transition_probabilities(cls):
        tp_a = np.zeros((cls.N_STATES, cls.N_STATES))
        tp_a[:, -1] = 1

        tp_b = np.full((cls.N_STATES, cls.N_STATES), 1/6)
        tp_b[:, -1] = 0

        return np.stack([tp_a, tp_b])

    def take_action(self, state: State, action: Action) -> State:
        target_probs = self.transition_probs[action.value, state.value]
        return np.random.choice(self.STATES, p=target_probs)



In [5]:
p = Problem()
p.transition_probs.shape

(2, 7, 7)

In [6]:
p.transition_probs_a


array([[0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [7]:
p.transition_probs_b

array([[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.16666667, 0.        ]])

In [8]:
for i in range(10):
    print(p.take_action(p.State.S1, Action.A).name, end=", ")

S7, S7, S7, S7, S7, S7, S7, S7, S7, S7, 

In [9]:
for i in range(10):
    print(p.take_action(p.State.S1, Action.B).name, end=", ")

S4, S4, S2, S5, S1, S2, S2, S6, S5, S2, 

In [10]:
p.transition_probs[0, 1]

array([0., 0., 0., 0., 0., 0., 1.])

In [11]:
np.random.choice(p.STATES, p=p.transition_probs[0, 1])

<State.S7: 6>

In [19]:
w_init = np.array(6*[1] + [10] + 8 * [1])
w_init

array([ 1,  1,  1,  1,  1,  1, 10,  1,  1,  1,  1,  1,  1,  1,  1])

In [20]:
def define_weight_matrices(n_states: int, n_params: int):
    phi_a = np.zeros((n_states, n_params), dtype=int)
    phi_a[:, n_params//2] = 1
    phi_a[np.arange(n_states-1), np.arange(n_states-1)] = 2
    phi_a[n_states-1, n_states-1] = 1
    phi_a[n_states-1, n_states] = 2

    phi_b = np.zeros((n_states, n_params), dtype=int)
    phi_b[np.arange(n_states), n_params//2 + n_params%2 + np.arange(n_states)] = 1

    return np.stack([phi_a, phi_b]).swapaxes(0, 1)


In [21]:
phi = define_weight_matrices(Problem.N_STATES, len(w))
phi.shape

(7, 2, 15)

In [22]:
phi[:, 0]

array([[2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0]])

In [23]:
phi[:, 1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [24]:
phi @ w_init

array([[ 3,  1],
       [ 3,  1],
       [ 3,  1],
       [ 3,  1],
       [ 3,  1],
       [ 3,  1],
       [12,  1]])

In [25]:
alpha = 0.01
gamma = 0.99
REWARD = 0

In [40]:
w = w_init[:].astype(float)
state = np.random.choice(p.STATES)
action = choose_action()
states_sequence = [state]
actions_sequence = []
w_norms = [np.linalg.norm(w)]
for i in range(20):
    actions_sequence.append(action)

    new_state = p.take_action(state, action)
    new_action = choose_action()

    phi_st = phi[state.value, action.value]
    phi_new_st = phi[new_state.value, new_action.value]
    estimate = phi_st @ w
    target = REWARD + gamma * phi_new_st @ w
    w += alpha * phi_st * (target - estimate)

    state = new_state
    action = new_action
    states_sequence.append(state)
    w_norms.append(np.linalg.norm(w))


In [43]:
for s, a in zip(states_sequence, actions_sequence):
    print(f"{s.name}, {a.name}, ", end='')
print(states_sequence[-1].name)

S7, B, S2, A, S7, B, S3, B, S5, B, S6, B, S6, B, S4, B, S6, B, S2, B, S6, B, S4, B, S4, B, S5, B, S1, B, S4, B, S3, B, S4, B, S2, B, S3, B, S3


In [45]:
np.array(w_norms)

array([10.67707825, 10.67894134, 10.67344084, 10.67341247, 10.6734031 ,
       10.67339373, 10.67338436, 10.67337509, 10.67336553, 10.67335635,
       10.67334671, 10.67333753, 10.67332816, 10.67331889, 10.67330962,
       10.67329997, 10.67329079, 10.67328115, 10.67327206, 10.6732626 ,
       10.67510735])

In [46]:
w_init

array([ 1,  1,  1,  1,  1,  1, 10,  1,  1,  1,  1,  1,  1,  1,  1])